# IBM Data Science Capstone Project

This Project is to create a model which can recommend the nearest neighborhood in canada to move to using machine learning alogrithms and geo api from foursquare

## Table of Content:
---

 * [Prepare & Clean The Data](#Cleaning-and-Data-Preparing)
 * [Visualize The Data](#)

# Cleaning and Data Preparing

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [27]:
data = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969")
df = data[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [28]:
# cleaning
df = df[df['Borough'] != 'Not assigned']
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [29]:
df[df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [30]:
df.shape

(103, 3)

### Import Data Corndinates

In [31]:
df_cor = pd.read_csv('https://cocl.us/Geospatial_data')

In [32]:
df_cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
df_cor.set_index('Postal Code')

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [34]:
p_data = df.join(df_cor.set_index('Postal Code'), on='Postal Code')
p_data.reset_index(drop = True, inplace=True)
p_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [35]:
CLIENT_ID = 'QMFYKU4KSWVFRMNFCPSMQR2Q3SBS55EGZNEQKKERK1G02530' # your Foursquare ID
CLIENT_SECRET = '2044ODGCFHSQ0WBXMIGYKXCOBG4FZIF13OAIAHJNUHK5Q00J' # your Foursquare Secret
ACCESS_TOKEN = 'RHKVHHQNZPLMSTJ3IEY5Q1JVKWEW5QIHNBZUEVQ2CEVAWOXW' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
radius = 500

In [36]:
def venues_(name, latitude, longitude, radius=500):
    
    venues_list = []
    
    for name, latitude, longitude in zip(name, latitude, longitude):

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, 
            latitude, longitude, 
            ACCESS_TOKEN, VERSION, 
            radius, LIMIT)
        
        response = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(name, latitude, longitude, v['venue']['name'],
                                v['venue']['location']['lat'],
                                v['venue']['location']['lng'],
                                v['venue']['categories'][0]['name']) for v in response])
        
    df_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    df_venues.columns=['Neighbourhood', 'Neighbourhood_lat', 'Neighbourhood_lng',
                         'Venue', 'Venue_lat', 'Venue_lng', 'Venue_cat']
    
    return(df_venues)

In [37]:
combiend_df = venues_(p_data['Neighbourhood'], p_data['Latitude'], p_data['Longitude'])

combiend_df.head()

,Neighbourhood,Neighbourhood_lat,Neighbourhood_lng,Venue,Venue_lat,Venue_lng,Venue_cat
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Careful & Reliable Painting,43.752622,-79.331957,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
3,Parkwoods,43.753259,-79.329656,Towns On The Ravine,43.754754,-79.332552,Hotel
4,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping


In [38]:
combiend_df.groupby('Neighbourhood').count()

,Neighbourhood_lat,Neighbourhood_lng,Venue,Venue_lat,Venue_lng,Venue_cat
Neighbourhood,,,,,,
Agincourt,7,7,7,7,7,7
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Wilson Heights, Downsview North",34,34,34,34,34,34
Bayview Village,6,6,6,6,6,6
"Bedford Park, Lawrence Manor East",52,52,52,52,52,52
...,...,...,...,...,...,...
"Willowdale, Willowdale West",9,9,9,9,9,9
Woburn,4,4,4,4,4,4
Woodbine Heights,16,16,16,16,16,16


In [39]:
len(combiend_df['Venue_cat'].unique())

318

In [40]:
combiend_df_onehot = pd.get_dummies(combiend_df[['Venue_cat']], prefix="", prefix_sep="")

combiend_df_onehot['Neighbourhood'] = combiend_df['Neighbourhood']

# move neighborhood file to the first column

neigh_col = [combiend_df_onehot.columns[-1]] + list(combiend_df_onehot.columns[:-1])
combiend_df_onehot = combiend_df_onehot[neigh_col]

combiend_df_onehot.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_df = combiend_df_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_df

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,"Alderwood, Long Branch",0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Bayview Village,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,"Bedford Park, Lawrence Manor East",0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.019231,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
94,Woburn,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
95,Woodbine Heights,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
96,York Mills West,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [42]:
def top_venues(data, num):
    df_venues = data.iloc[1:]
    df_venues_sorted = df_venues.sort_values(ascending=False)
    
    return df_venues_sorted.index.values[0: num]

In [43]:
num_venues = 10

col_ind = ['st', 'nd', 'rd']

cols = ['Neighbourhood']
for i in np.arange(num_venues):
    try:
        cols.append('{}{} Common Venues'.format(i+1, col_ind[i]))
    except:
        cols.append('{}th Common Venues'.format(i+1))
        
neigh_venues_sorted = pd.DataFrame(columns=cols)
neigh_venues_sorted['Neighbourhood'] = toronto_df['Neighbourhood']

for i in np.arange(toronto_df.shape[0]):
    neigh_venues_sorted.iloc[i, 1:] = top_venues(toronto_df.iloc[i, :], num_venues)
    
    
neigh_venues_sorted.head()

,Neighbourhood,1st Common Venues,2nd Common Venues,3rd Common Venues,4th Common Venues,5th Common Venues,6th Common Venues,7th Common Venues,8th Common Venues,9th Common Venues,10th Common Venues
0,Agincourt,Latin American Restaurant,Hardware Store,Fireworks Store,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Peking Duck Restaurant,Pastry Shop,Music Venue
1,"Alderwood, Long Branch",Pizza Place,Pub,Skating Rink,Sandwich Place,Athletics & Sports,Coffee Shop,Gym,Pharmacy,Playground,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Pharmacy,Mobile Phone Shop,Ice Cream Shop,Spa,Coffee Shop,Bank,Sushi Restaurant,Supermarket,Gas Station,Frozen Yogurt Shop
3,Bayview Village,Spa,Chinese Restaurant,Bank,Gym,Japanese Restaurant,Café,New American Restaurant,Neighborhood,Nail Salon,Music Venue
4,"Bedford Park, Lawrence Manor East",Spa,Pizza Place,Italian Restaurant,Juice Bar,Business Service,Sushi Restaurant,Massage Studio,Boutique,Coffee Shop,Sandwich Place


In [44]:
toronto_df_clusters = toronto_df.drop('Neighbourhood', 1)

clusters = KMeans(n_clusters=5, random_state=0).fit(toronto_df_clusters)

clusters.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [45]:
neigh_venues_sorted.insert(0, 'Cluster Labels', clusters.labels_)

toronto_data = p_data

toronto_data = toronto_data.join(neigh_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='right')

toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Common Venues,2nd Common Venues,3rd Common Venues,4th Common Venues,5th Common Venues,6th Common Venues,7th Common Venues,8th Common Venues,9th Common Venues,10th Common Venues
78,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Latin American Restaurant,Hardware Store,Fireworks Store,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Peking Duck Restaurant,Pastry Shop,Music Venue
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1,Pizza Place,Pub,Skating Rink,Sandwich Place,Athletics & Sports,Coffee Shop,Gym,Pharmacy,Playground,Dance Studio
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,1,Pharmacy,Mobile Phone Shop,Ice Cream Shop,Spa,Coffee Shop,Bank,Sushi Restaurant,Supermarket,Gas Station,Frozen Yogurt Shop
39,M2K,North York,Bayview Village,43.786947,-79.385975,1,Spa,Chinese Restaurant,Bank,Gym,Japanese Restaurant,Café,New American Restaurant,Neighborhood,Nail Salon,Music Venue
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,1,Spa,Pizza Place,Italian Restaurant,Juice Bar,Business Service,Sushi Restaurant,Massage Studio,Boutique,Coffee Shop,Sandwich Place


In [46]:
address = 'Toronto'

geolocator = Nominatim(user_agent='explorer')

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [54]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters